In [ ]:
import requests

In [ ]:
import docx
import spacy
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
# Load spaCy Russian tokenizer
nlp = spacy.blank("ru")

# Load a pre-trained model tokenizer and model from Hugging Face transformers
model_name = "bert-base-multilingual-cased"  # Pre-trained multilingual BERT model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def read_docx(file_path):
    # Open the Word document
    doc = docx.Document(file_path)

    # Extract text content from the document
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)

    # Join all paragraphs into a single string
    document_text = "\n".join(full_text)
    return document_text

def chunk_and_encode_document(document_text, max_chunk_len=200):
    # Split the document into chunks
    doc = nlp(document_text)
    chunks = []
    current_chunk = []

    for token in doc:
        if len(current_chunk) + len(token.text) > max_chunk_len:
            chunks.append(" ".join(current_chunk))
            current_chunk = []

        current_chunk.append(token.text)

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    # Encode each chunk into vectors
    encoded_chunks = []

    for chunk in chunks:
        # Tokenize the chunk
        inputs = tokenizer(chunk, return_tensors="pt", padding=True, truncation=True)

        # Pass the input through the model and get the hidden states
        with torch.no_grad():
            outputs = model(**inputs)

        # Extract the embeddings (CLS token embedding)
        embeddings = outputs.last_hidden_state[:, 0, :]

        # Normalize the embeddings
        normalized_embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)

        # Append the normalized embedding to the list
        encoded_chunks.append(normalized_embeddings)

    return chunks, encoded_chunks

# Example usage
file_path = "main_doc.docx"
document_text = read_docx(file_path)
chunks, encoded_chunks = chunk_and_encode_document(document_text)


In [ ]:
def encode_query(query_text):
    # Encode the query text into a vector
    inputs = tokenizer(query_text, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the embeddings (CLS token embedding)
    query_embedding = outputs.last_hidden_state[:, 0, :]

    # Normalize the embedding
    normalized_query_embedding = torch.nn.functional.normalize(query_embedding, p=2, dim=1)

    return normalized_query_embedding



def vector_search(query_embedding, encoded_chunks, chunks):
    # Compute cosine similarity between query embedding and all chunk embeddings
    similarities = []

    for emb in encoded_chunks:
        sim = cosine_similarity(query_embedding.numpy().reshape(1, -1), emb.numpy().reshape(1, -1))
        similarities.append(sim.item())

    # Find indices of the top 3 most similar chunks
    closest_indices = np.argsort(similarities)[::-1][:3]
    closest_chunks = [chunks[idx] for idx in closest_indices]

    return closest_chunks



query = 'Какова роль академического руководителя в НИУ ВШЭ?'
query_embedding = encode_query(query)
closest_chunks = vector_search(query_embedding, encoded_chunks, chunks)
i=1
print("Closest chunks to the query:")
for idx, chunk in enumerate(closest_chunks, start=1):
    print('chunk ',i)
    print()
    i+=1
    print(f"{idx}. {chunk}")
    print()

Closest chunks to the query:
chunk  1

1. рассчитывается только после повторной сдачи таких Элементов . Порядок организации таких повторных сдач отражается в ПУД . Повторные сдачи по таким блокирующим Элементам проводятся по КИМам и технологии , аналогичным применяемым при первом проведении этого Элемента контроля . Порядок проведения таких повторных сдач во время сессии в случае потенциального конфликта с основным расписанием сессии регулируется пунктом 57 Положения . 
 Если у студента возникает неудовлетворительная оценка по Элементу контроля по Дисциплине , то она не может быть причиной недопуска к следующим Элементам контроля ( включая экзамены ) по этой Дисциплине , проводимым до окончания ближайшей сессии , в которую предусмотрена промежуточная аттестация по этой Дисциплине . 
 Преподаватель имеет право незначительно корректировать количество и процедуры организации Элементов контроля . В этом случае он должен зафиксировать изменения в ПУД посредством Конструктора ПУД и отдельно 

In [ ]:


prompt = {
    "modelUri": "gpt://b1g91rv9466rln6hv7lr/yandexgpt-lite",
    "completionOptions": {
        "stream": False,
        "temperature": 0.6,
        "maxTokens": "2000"
    },
    "messages": [
        {
            "role": "system",
            "text": "ты умный помошник студента"
        },
        {
            "role": "user",
            "text": "Привет,......?"
        }
    ]
}


url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
headers = {
    "Content-Type": "application/json",
    "Authorization": "________________________"
}

response = requests.post(url, headers=headers, json=prompt)
result = response.text
print(result)

{"result":{"alternatives":[{"message":{"role":"assistant","text":"Для написания любой работы, включая работу на соискание степени LLM (магистра права), необходимо следовать определённым стандартам и требованиям, которые могут различаться в зависимости от учебного заведения и программы. Вот некоторые общие рекомендации, которые могут вам помочь:\n1. **Определите тему и фокус вашей работы.** LLM-программы обычно требуют глубокого анализа и исследования в определённой области права. Тема должна быть актуальной и соответствовать вашей области специализации.\n\n2. **Определите цель и задачи работы.** Что вы хотите достичь в своей работе? Какие вопросы вы хотите рассмотреть? Это поможет вам организовать структуру и содержание вашей работы.\n3. **Проведите исследование.** Соберите информацию из различных источников, таких как книги, статьи, юридические прецеденты и другие релевантные материалы. Убедитесь, что вы используете надёжные и авторитетные источники.\n4. **Разработайте план работы.** 